In [ ]:
import os
from pathlib import Path
from operator import itemgetter
import pickle
import pydicom
from time import time
import pandas

import matplotlib.pyplot as plt
import numpy as np

from catch_converter.parse_contours import parse_cvi42ws
from LazyLuna.Mini_LL import *
from LazyLuna.CATCH_utils import *
from LazyLuna.Tables  import *
from LazyLuna.Figures import *

import shapely

In [ ]:
bp        = '/Users/dietrichhadler/Desktop/Daten/LGE'
bp_annos1 = '/Users/dietrichhadler/Desktop/Daten/LGE/Max'
bp_annos2 = '/Users/dietrichhadler/Desktop/Daten/LGE/MaxIntra'
bp_annos3 = '/Users/dietrichhadler/Desktop/Daten/LGE/TGrandy'
bp_cases  = '/Users/dietrichhadler/Desktop/Daten/LGE/Cases'
bp_imgs   = '/Users/dietrichhadler/Desktop/Daten/LGE/Imgs'

In [ ]:
"""
# unpack the workspaces from ws_path to case_storage_path
for bp_annos in [bp_annos1,bp_annos2,bp_annos3]:
    #bp_annos = bp_annos3
    parse_cvi42ws(bp_annos, bp_annos, process=True, debug=False)
    cases = []
    paths = get_imgs_and_annotation_paths(bp_imgs, bp_annos)
    view = SAX_LGE_View()
    for imgp, annop in paths:
        #if '024' not in imgp: continue
        try:
            print(imgp, annop)
            st = time()
            case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(bp_annos))
            view.initialize_case(case, debug=True)
            case.store(bp_cases)
            cases.append(case)
            print('Case customization took: ', time()-st, 'Case: ', case.reader_name, case.case_name)
        except Exception as e:
            print('Failed at: ', imgp, annop, ' , Exception: ', e)
"""

In [ ]:
class SAX_LGE_View(View):
    def __init__(self):
        self.colormap            = 'gray'
        self.available_colormaps = ['gray']
        self.load_categories()
        # contour names with scars
        self.contour_names = ['lv_endo', 'lv_myo']
        for exclude in [False, True]:
            for highres in [False, True]:
                cont_name = 'scar_fwhm' + ('_res_8' if highres else '') + ('_excluded_area' if exclude else '')
                self.contour_names += [cont_name]
        self.contour2categorytype = {cname:self.all for cname in self.contour_names}
        
        # register tabs here:
        from LazyLuna.Guis.Addable_Tabs.CC_Metrics_Tab                        import CC_Metrics_Tab
        from LazyLuna.Guis.Addable_Tabs.CCs_ClinicalResults_Tab               import CCs_ClinicalResults_Tab
        self.case_tabs  = {'Metrics and Figure': CC_Metrics_Tab}
        self.stats_tabs = {'Clinical Results'  : CCs_ClinicalResults_Tab}
        
    def load_categories(self):
        self.all = [SAX_LGE_Category]

    def get_categories(self, case, contour_name=None):
        types = [c for c in self.contour2categorytype[contour_name]]
        cats  = [c for c in case.categories if type(c) in types]
        return cats

    def initialize_case(self, case, debug=False):
        if debug: st=time()
        # switch images
        case.imgs_sop2filepath = case.all_imgs_sop2filepath['SAX LGE']
        # attach annotation type
        case.attach_annotation_type(SAX_LGE_Annotation)
        # if categories have not been attached, attach the first and init other_categories
        # otherwise it has categories and a type, so store the old categories for later use
        if not hasattr(case, 'other_categories'): case.other_categories = dict()
        case.attach_categories([SAX_LGE_Category])
        # A SCAR calculating preprocessing step is necessary for LGE
        if debug: st_preprocess = time()
        cat = case.categories[0]; cat.preprocess_scars()
        if debug: print('Calculating scars took: ', time()-st_preprocess)
        if debug: print('Set of anno keys are: ', list(set([akey for a in cat.get_annos() for akey in a.anno.keys()])))
        case.other_categories['SAX LGE'] = case.categories
        case.categories = []
        if debug: print('Case categories are: ', case.categories)
        
        # set new type
        case.type = 'SAX LGE'
        case.available_types.add('SAX LGE')
        if debug: print('Customization in SAX LGE view took: ', time()-st)
        return case
    
    def customize_case(self, case, debug=False):
        if debug: st=time()
        # switch images
        case.imgs_sop2filepath = case.all_imgs_sop2filepath['SAX LGE']
        # attach annotation type
        case.attach_annotation_type(SAX_LGE_Annotation)
        # if categories have not been attached, attach the first and init other_categories
        # otherwise it has categories and a type, so store the old categories for later use
        if not hasattr(case, 'categories'):
            case.other_categories = dict()
            case.attach_categories([SAX_LGE_Category])
            case.other_categories['SAX LGE'] = case.categories
        else:
            if 'SAX LGE' in case.other_categories.keys(): case.categories = case.other_categories['SAX LGE']
            else: case.attach_categories([SAX_LGE_Category])
        if debug: print('Case categories are: ', case.categories)
        # attach CRs
        case.attach_clinical_results([SAXLGE_LVV,       SAXLGE_LVMYOV, 
                                      SAXLGE_LVMYOMASS, SAXLGE_SCARVOL,
                                      SAXLGE_SCARMASS,  SAXLGE_SCARF,
                                      SAXLGE_EXCLVOL,   SAXLGE_EXCLMASS])
        # set new type
        case.type = 'SAX LGE'
        if debug: print('Customization in SAX LGE view took: ', time()-st)
        return case

In [ ]:
class SAX_LGE_Category:
    def __init__(self, case, debug=False):
        self.case = case
        self.sop2depthandtime = self.get_sop2depthandtime(case.imgs_sop2filepath, debug=debug)
        self.depthandtime2sop = {v:k for k,v in self.sop2depthandtime.items()}
        self.set_nr_slices_phases()
        self.set_image_height_width_depth()
        self.name = 'SAX LGE'
        self.phase = 0

    def get_sop2depthandtime(self, sop2filepath, debug=False):
        if debug: st = time()
        # returns dict sop --> (depth, time)
        imgs = {k:pydicom.dcmread(sop2filepath[k]) for k in sop2filepath.keys()}
        sortable_slice_location = [float(v.SliceLocation) for sopinstanceuid, v in imgs.items()]
        sl_len = len(set([elem for elem in sortable_slice_location]))
        sorted_slice_location = np.array(sorted(sortable_slice_location))
        sop2depthandtime = dict()
        for sopinstanceuid in imgs.keys():
            s_loc = imgs[sopinstanceuid].SliceLocation
            for i in range(len(sorted_slice_location)):
                if not s_loc==sorted_slice_location[i]: continue
                sop2depthandtime[sopinstanceuid] = (i,0)
        # potentially flip slice direction: base top x0<x1, y0>y1, z0>z1, apex top x0>x1, y0<y1, z0<z1
        depthandtime2sop = {v:k for k,v in sop2depthandtime.items()}
        img1, img2 = imgs[depthandtime2sop[(0,0)]], imgs[depthandtime2sop[(1,0)]]
        img1x,img1y,img1z = list(map(float,img1.ImagePositionPatient))
        img2x,img2y,img2z = list(map(float,img2.ImagePositionPatient))
        if img1x<img2x and img1y>img2y and img1z>img2z: pass
        else: #img1x>img2x and img1y<img2y and img1z<img2z:
            max_depth = sl_len-1
            for sop in sop2depthandtime.keys():
                sop2depthandtime[sop] = (max_depth-sop2depthandtime[sop][0], 0)
        if debug: print('calculating sop2sorting takes: ', time()-st)
        return sop2depthandtime

    def set_image_height_width_depth(self, debug=False):
        if debug: st = time()
        nr_slices = self.nr_slices
        dcm1 = self.case.load_dcm(self.depthandtime2sop[(0, 0)])
        dcm2 = self.case.load_dcm(self.depthandtime2sop[(1, 0)])
        self.height, self.width = dcm1.pixel_array.shape
        self.pixel_h, self.pixel_w = list(map(float, dcm1.PixelSpacing))
        spacingbs = []
        for d in range(nr_slices-1):
            dcm1 = self.case.load_dcm(self.depthandtime2sop[(d,   0)])
            dcm2 = self.case.load_dcm(self.depthandtime2sop[(d+1, 0)])
            spacingbs += [round(np.abs(dcm1.SliceLocation - dcm2.SliceLocation), 2)]
            try: self.slice_thickness = dcm1.SliceThickness
            except Exception as e: print('Exception in SAX_LGE_Category, ', e)
        self.spacing_between_slices = min(spacingbs)
        print('Spacings: ', spacingbs)
        self.missing_slices = []
        for d in range(nr_slices-1):
            dcm1 = self.case.load_dcm(self.depthandtime2sop[(d,   0)])
            dcm2 = self.case.load_dcm(self.depthandtime2sop[(d+1, 0)])
            curr_spacing = round(np.abs(dcm1.SliceLocation - dcm2.SliceLocation), 2)
            if round(curr_spacing / self.spacing_between_slices) != 1:
                for m in range(int(round(curr_spacing / self.spacing_between_slices))-1):
                    self.missing_slices += [(d + m)]
                
    def set_nr_slices_phases(self):
        dat = list(self.depthandtime2sop.keys())
        self.nr_phases = 1
        self.nr_slices = max(dat, key=itemgetter(0))[0]+1
        
    def get_dcm(self, slice_nr, phase_nr):
        sop = self.depthandtime2sop[(slice_nr, phase_nr)]
        return self.case.load_dcm(sop)

    def get_anno(self, slice_nr, phase_nr=0):
        sop = self.depthandtime2sop[(slice_nr, phase_nr)]
        return self.case.load_anno(sop)

    def get_img(self, slice_nr, phase_nr=0, value_normalize=True, window_normalize=False):
        sop = self.depthandtime2sop[(slice_nr, phase_nr)]
        return self.case.get_img(sop, value_normalize=value_normalize, window_normalize=window_normalize)

    def get_imgs(self, value_normalize=True, window_normalize=False):
        return [self.get_img(d, 0, value_normalize, window_normalize) for d in range(self.nr_slices)]

    def get_annos(self):
        return [self.get_anno(d,0) for d in range(self.nr_slices)]

    def get_anno_with_scar_fwhm(self, slice_nr, exclude=True, highres=True):
        img  = self.get_img (slice_nr, 0)
        anno = self.get_anno(slice_nr, 0)
        if anno.has_contour('saEnhancementReferenceMyoContour'):
            anno.add_fwhm_scar(img, exclude=exclude, highres=highres)
            return anno
        for i in range(self.nr_slices):
            low_d  = max(0, slice_nr - i)
            high_d = min(slice_nr + i, self.nr_slices-1)
            for d in [low_d, high_d]:
                other_anno = self.get_anno(d, 0)
                if other_anno.has_contour('saEnhancementReferenceMyoContour'):
                    other_img = self.get_img(d, 0)
                    other_anno.add_fwhm_scar(other_img, exclude=exclude, highres=highres)
                    anno.add_fwhm_scar_other_slice(img, other_anno, exclude=exclude, highres=highres)
                    return anno
    
    def preprocess_scars(self):
        for d in range(self.nr_slices):
            anno = self.get_anno(d,0)
            if anno.sop not in self.case.annos_sop2filepath: continue
            new_anno = anno.anno
            for exclude in [False, True]:
                for highres in [False, True]:
                    anno = self.get_anno_with_scar_fwhm(d, exclude, highres)
                    new_anno = {**new_anno, **anno.anno}
            path = self.case.annos_sop2filepath[anno.sop]
            pickle.dump(new_anno, open(path, 'wb'), pickle.HIGHEST_PROTOCOL)
    
    def get_base_apex(self, cont_name, debug=False):
        annos     = self.get_annos()
        has_conts = [a.has_contour(cont_name) for a in annos]
        if True not in has_conts: return 0
        base_idx = has_conts.index(True)
        apex_idx = self.nr_slices - has_conts[::-1].index(True) - 1
        if debug: print('Base idx / Apex idx: ', base_idx, apex_idx)
        return base_idx, apex_idx
    
    def get_volume(self, cont_name, debug=False):
        annos = self.get_annos()
        pixel_area = self.pixel_h * self.pixel_w
        areas = [a.get_contour(cont_name).area*pixel_area if a is not None else 0.0 for a in annos]
        if debug: print('Areas: ', [round(a, 2) for a in areas])
        has_conts = [a!=0 for a in areas]
        if True not in has_conts: return 0
        base_idx = has_conts.index(True)
        apex_idx = self.nr_slices - has_conts[::-1].index(True) - 1
        if debug: print('Base idx / Apex idx: ', base_idx, apex_idx)
        vol = 0
        for d in range(self.nr_slices):
            pixel_depth = (self.slice_thickness+self.spacing_between_slices)/2.0 if d in [base_idx, apex_idx] else self.spacing_between_slices
            vol += areas[d] * pixel_depth
        # for missing slices
        for d in self.missing_slices:
            pixel_depth = self.spacing_between_slices
            vol += (areas[d] + areas[d+1])/2 * pixel_depth
        return vol / 1000.0

In [ ]:
class SAX_LGE_Annotation(Annotation):
    def __init__(self, sop, filepath):
        super().__init__(sop, filepath)

    def set_information(self):
        self.name = 'SAX LGE Annotation'
        self.contour_names = ['lv_endo', 'lv_epi', 'lv_myo',
                              'excludeEnhancementAreaContour',
                              'saEnhancementReferenceMyoContour',
                              'scar_fwhm', 'scar_fwhm_res_8', 'scar_fwhm_excluded_area', 'scar_fwhm_res_8_excluded_area']
        
        self.point_names   = ['sacardialRefPoint']
        self.pixel_h, self.pixel_w = self.anno['info']['pixelSize'] if 'info' in self.anno.keys() and 'pixelSize' in self.anno['info'].keys() else (-1,-1)#1.98,1.98
        self.h,       self.w       = self.anno['info']['imageSize'] if 'info' in self.anno.keys() and 'imageSize' in self.anno['info'].keys() else (-1,-1)

    def add_fwhm_scar(self, img, exclude=True, highres=True, normalize_myocardium=True):
        if not self.has_contour('saEnhancementReferenceMyoContour'): return
        h, w  = img.shape
        cont  = self.get_contour('saEnhancementReferenceMyoContour')
        mask  = to_mask(cont, h, w)
        self.scar_max = img[np.where(mask!=0)].max()
        #threshold = myoMinimum + (myoMax-Min) / 2
        thresh = self.scar_max/2.0
        if normalize_myocardium:
            mask      = to_mask(self.get_contour('lv_myo'), h, w)
            lvmyo_min = img[np.where(mask!=0)].min()
            thresh    = lvmyo_min + (self.scar_max - lvmyo_min)/2.0
            #print('Possible threshs: ', self.scar_max/2.0, thresh)
        scale = 8.0 if highres else 1.0
        img  = cv2.resize(img, (int(np.round(w*scale)), int(np.round(h*scale))), interpolation=cv2.INTER_CUBIC)
        cont = to_polygon((img>thresh).astype(np.int16))
        cont = shapely.affinity.scale(cont, xfact=1.0/scale, yfact=1.0/scale, origin=(0,0))
        cont = cont.intersection(self.get_contour('lv_myo'))
        if exclude: cont = cont.difference(self.get_contour('excludeEnhancementAreaContour'))
        cont = geometry_collection_to_Polygon(cont)
        cont_name = 'scar_fwhm' + ('_res_8' if highres else '') + ('_excluded_area' if exclude else '')
        self.anno[cont_name] = {'cont':cont, 'contType':'FREE', 'subpixelResolution': scale}
        self.contour_names += [cont_name]
            
    def add_fwhm_scar_other_slice(self, img, other_anno, exclude=True, highres=True, normalize_myocardium=True):
        h, w  = img.shape
        self.scar_max    = other_anno.scar_max
        #threshold = myoMinimum + (myoMax-Min) / 2
        thresh = self.scar_max/2.0
        if normalize_myocardium and self.has_contour('lv_myo'):
            mask      = to_mask(self.get_contour('lv_myo'), h, w)
            lvmyo_min = img[np.where(mask!=0)].min()
            lvmyo_max = img[np.where(mask!=0)].max()
            thresh    = lvmyo_min + (self.scar_max - lvmyo_min)/2.0
            #print('Possible threshs other slice: ', self.scar_max/2.0, thresh)
        scale = 8.0 if highres else 1.0
        img   = cv2.resize(img, (int(np.round(w*scale)), int(np.round(h*scale))), interpolation=cv2.INTER_CUBIC)
        cont  = to_polygon((img>thresh).astype(np.int16))
        cont  = shapely.affinity.scale(cont, xfact=1.0/scale, yfact=1.0/scale, origin=(0,0))
        cont  = cont.intersection(self.get_contour('lv_myo'))
        if exclude: cont = cont.difference(self.get_contour('excludeEnhancementAreaContour'))
        cont  = geometry_collection_to_Polygon(cont)
        if cont.is_empty: return
        cont_name = 'scar_fwhm' + ('_res_8' if highres else '') + ('_excluded_area' if exclude else '')
        self.anno[cont_name] = {'cont':cont, 'contType':'FREE', 'subpixelResolution': scale}
        self.contour_names += [cont_name]

In [ ]:
class SAXLGE_LVV(Clinical_Result):
    def __init__(self, case):
        self.case = case
        self.set_CR_information()
    def set_CR_information(self):
        self.name = 'LVV'
        self.unit = '[ml]'
        #self.cat  = [c for c in self.case.categories if isinstance(c, SAX_LGE_Category)][0]
        self.cat  = self.case.categories[0]
    def get_cr(self, string=False):
        cr = self.cat.get_volume('lv_endo')
        return "{:.2f}".format(cr) if string else cr
    def get_cr_diff(self, other, string=False):
        cr_diff = self.get_cr()-other.get_cr()
        return "{:.2f}".format(cr_diff) if string else cr_diff
    
class SAXLGE_LVMYOMASS(Clinical_Result):
    def __init__(self, case):
        self.case = case
        self.set_CR_information()
    def set_CR_information(self):
        self.name = 'LVMMASS'
        self.unit = '[g]'
        #self.cat  = [c for c in self.case.categories if isinstance(c, SAX_LGE_Category)][0]
        self.cat  = self.case.categories[0]
    def get_cr(self, string=False):
        cr = 1.05 * self.cat.get_volume('lv_myo')
        return "{:.2f}".format(cr) if string else cr
    def get_cr_diff(self, other, string=False):
        cr_diff = self.get_cr()-other.get_cr()
        return "{:.2f}".format(cr_diff) if string else cr_diff

class SAXLGE_LVMYOV(Clinical_Result):
    def __init__(self, case):
        self.case = case
        self.set_CR_information()
    def set_CR_information(self):
        self.name = 'LVMV'
        self.unit = '[ml]'
        #self.cat  = [c for c in self.case.categories if isinstance(c, SAX_LGE_Category)][0]
        self.cat  = self.case.categories[0]
    def get_cr(self, string=False):
        cr = self.cat.get_volume('lv_myo')
        return "{:.2f}".format(cr) if string else cr
    def get_cr_diff(self, other, string=False):
        cr_diff = self.get_cr()-other.get_cr()
        return "{:.2f}".format(cr_diff) if string else cr_diff

class SAXLGE_SCARMASS(Clinical_Result):
    def __init__(self, case):
        self.case = case
        self.set_CR_information()
    def set_CR_information(self):
        self.name = 'SCARM'
        self.unit = '[g]'
        #self.cat  = [c for c in self.case.categories if isinstance(c, SAX_LGE_Category)][0]
        self.cat  = self.case.categories[0]
    def get_cr(self, string=False):
        #cr = 1.05 * self.cat.get_volume('scar_fwhm_res_8_excluded_area')
        cr = 1.05 * self.cat.get_volume('scar_fwhm_excluded_area')
        return "{:.2f}".format(cr) if string else cr
    def get_cr_diff(self, other, string=False):
        cr_diff = self.get_cr()-other.get_cr()
        return "{:.2f}".format(cr_diff) if string else cr_diff

class SAXLGE_SCARVOL(Clinical_Result):
    def __init__(self, case):
        self.case = case
        self.set_CR_information()
    def set_CR_information(self):
        self.name = 'SCARV'
        self.unit = '[ml]'
        #self.cat  = [c for c in self.case.categories if isinstance(c, SAX_LGE_Category)][0]
        self.cat  = self.case.categories[0]
    def get_cr(self, string=False):
        #cr = self.cat.get_volume('scar_fwhm_res_8_excluded_area')
        cr = self.cat.get_volume('scar_fwhm_excluded_area')
        return "{:.2f}".format(cr) if string else cr
    def get_cr_diff(self, other, string=False):
        cr_diff = self.get_cr()-other.get_cr()
        return "{:.2f}".format(cr_diff) if string else cr_diff
    
class SAXLGE_SCARF(Clinical_Result):
    def __init__(self, case):
        self.case = case
        self.set_CR_information()
    def set_CR_information(self):
        self.name = 'SCARF'
        self.unit = '[%]'
        #self.cat  = [c for c in self.case.categories if isinstance(c, SAX_LGE_Category)][0]
        self.cat  = self.case.categories[0]
    def get_cr(self, string=False):
        #scar = self.cat.get_volume('scar_fwhm_res_8_excluded_area')
        scar = self.cat.get_volume('scar_fwhm_excluded_area')
        lvm  = self.cat.get_volume('lv_myo')
        cr = 100.0 * (scar/(lvm+10**-9))
        return "{:.2f}".format(cr) if string else cr
    def get_cr_diff(self, other, string=False):
        cr_diff = self.get_cr()-other.get_cr()
        return "{:.2f}".format(cr_diff) if string else cr_diff
    
class SAXLGE_EXCLMASS(Clinical_Result):
    def __init__(self, case):
        self.case = case
        self.set_CR_information()
    def set_CR_information(self):
        self.name = 'EXCLMASS'
        self.unit = '[g]'
        #self.cat  = [c for c in self.case.categories if isinstance(c, SAX_LGE_Category)][0]
        self.cat  = self.case.categories[0]
    def get_cr(self, string=False):
        #scar_excl = self.cat.get_volume('scar_fwhm_res_8_excluded_area')
        #scar      = self.cat.get_volume('scar_fwhm_res_8')
        scar_excl = self.cat.get_volume('scar_fwhm_excluded_area')
        scar      = self.cat.get_volume('scar_fwhm')
        cr = 1.05 * (scar - scar_excl)
        return "{:.2f}".format(cr) if string else cr
    def get_cr_diff(self, other, string=False):
        cr_diff = self.get_cr()-other.get_cr()
        return "{:.2f}".format(cr_diff) if string else cr_diff
    
    
class SAXLGE_EXCLVOL(Clinical_Result):
    def __init__(self, case):
        self.case = case
        self.set_CR_information()
    def set_CR_information(self):
        self.name = 'EXCLVOL'
        self.unit = '[ml]'
        #self.cat  = [c for c in self.case.categories if isinstance(c, SAX_LGE_Category)][0]
        self.cat  = self.case.categories[0]
    def get_cr(self, string=False):
        scar_excl = self.cat.get_volume('scar_fwhm_res_8_excluded_area')
        scar      = self.cat.get_volume('scar_fwhm_res_8')
        cr = scar - scar_excl
        return "{:.2f}".format(cr) if string else cr
    def get_cr_diff(self, other, string=False):
        cr_diff = self.get_cr()-other.get_cr()
        return "{:.2f}".format(cr_diff) if string else cr_diff

In [ ]:
cases = [pickle.load(open(os.path.join(bp_cases,p), 'rb')) for p in os.listdir(bp_cases)]
cases1 = sorted([c for c in cases if c.reader_name=='Max'], key=lambda c:c.case_name)
cases2 = sorted([c for c in cases if c.reader_name=='TGrandy'], key=lambda c:c.case_name)
cases3 = sorted([c for c in cases if c.reader_name=='MaxIntra'], key=lambda c:c.case_name)
print([c.case_name[-8:-5] for c in cases1])
print([c.case_name[-8:-5] for c in cases2])
print([c.case_name[-8:-5] for c in cases3])
print(len(cases1))
print(len(cases2))
print(len(cases3))

v = SAX_LGE_View()
cases1 = [v.customize_case(c) for c in cases1]
cases2 = [v.customize_case(c) for c in cases2]
cases3 = [v.customize_case(c) for c in cases3]
print([c.categories[0].spacing_between_slices for c in cases1])
print([c.categories[0].spacing_between_slices for c in cases2])
print([c.categories[0].spacing_between_slices for c in cases3])
for c in cases2:
    cat = c.categories[0]
    print(c.case_name, cat.get_volume('lv_myo'))
    print(cat.get_base_apex('lv_myo'))

In [ ]:
for case1, case2 in zip(cases1, cases2):
    print(case1.case_name, case1.case_name==case2.case_name)
    print(case1.reader_name, case2.reader_name)
    if '013' not in case2.case_name: continue
    
    cat = case1.categories[0]
    cat2 = case2.categories[0]
    print(cat.slice_thickness)
    print(cat.spacing_between_slices)
    print(cat.pixel_h*cat.pixel_w)
    print(cat.get_volume('lv_myo'))
    print(cat.get_volume('lv_myo', 1))
    
    for cr1,cr2 in zip(case1.crs, case2.crs):
        print(cr1.name, cr1.get_cr(string=True), cr2.get_cr(string=True), cr1.get_cr_diff(cr2, string=True))
    print()
    for d in range(cat.nr_slices):
        img  = cat.get_img(d, 0, True, True)
        anno = cat.get_anno(d, 0)
        anno2 = cat2.get_anno(d, 0)
        a = cat.pixel_h*cat.pixel_w
        print(d, a*anno.get_contour('lv_myo').area, a*anno2.get_contour('lv_myo').area)
        continue
        fig, ax = plt.subplots(1,1, figsize=(17,17))
        h,w = img.shape
        ax.imshow(img, cmap='gray', extent=(0,w,h,0))
        #anno.plot_all_contour_outlines(ax)
        anno.plot_contour_outlines(ax, 'lv_myo')
        anno.plot_contour_outlines(ax, 'scar_fwhm')
        anno.plot_contour_face(ax, 'scar_fwhm_excluded_area', c='g')
        anno.plot_contour_face(ax, 'excludeEnhancementAreaContour', c='r')
        #anno.plot_contour_face(ax, 'saEnhancementReferenceMyoContour', c='g')
        plt.show()
    #break

In [ ]:
crs1 = dict()
crs2 = dict()
for case1, case2 in zip(cases1, cases2):
    print(case1.case_name)
    for case in [case1, case2]: v.customize_case(case)
    if case1.case_name=='3DCS_LV_002_034Y':
        for cr in case1.crs: 
            crs1[cr.name]=[]
            crs2[cr.name]=[]
    for cr1,cr2 in zip(case1.crs, case2.crs):
        crs1[cr1.name].append(cr1.get_cr())
        crs2[cr1.name].append(cr2.get_cr())

def bland_altman_plot(data1, data2, title):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0)            # Standard deviation of the difference
    plt.title(title)
    plt.scatter(mean, diff)
    plt.axhline(md,           color='gray', linestyle='--')
    plt.axhline(md + 1.96*sd, color='gray', linestyle='--')
    plt.axhline(md - 1.96*sd, color='gray', linestyle='--')
    plt.show()
    
def correlation_plot(data1, data2, title):
    print(title)
    print(np.round(data1).astype(int))
    print(np.round(data2).astype(int))
    print(np.round((np.asarray(data1)-np.asarray(data2))).astype(int).tolist())
    plt.scatter(data1, data2)
    xymax = np.max(data1+data2)
    plt.xlim(0, xymax)
    plt.ylim(0, xymax)
    plt.title(title)
    plt.show()

for k in crs1.keys():
    bland_altman_plot(crs1[k], crs2[k], k)
    correlation_plot(crs1[k], crs2[k], k)

In [ ]:
myo_dices  = []
scar_dices = []
areas1     = []
areas2     = []
for case1, case2 in zip(cases1, cases3):
    print(case1.case_name)
    print(case1.reader_name, case2.reader_name)
    for case in [case1, case2]: v.customize_case(case)
    cat1, cat2 = case1.categories[0], case2.categories[0]
    for d in range(cat1.nr_slices):
        img1,  img2  = cat1.get_img (d,0,True,True), cat2.get_img (d,0,True,True)
        anno1, anno2 = cat1.get_anno(d,0), cat2.get_anno(d,0)
        print(d)
        
        myo_dices.append(dice(anno1.get_contour('lv_myo'), anno2.get_contour('lv_myo')))
        scar_dices.append(dice(anno1.get_contour('scar_fwhm_excluded_area'), anno2.get_contour('scar_fwhm_excluded_area')))
        areas1.append(anno1.get_contour('scar_fwhm_excluded_area').area)
        areas2.append(anno2.get_contour('scar_fwhm_excluded_area').area)
        print('lvmyo dice: ', myo_dices [-1])
        print('scar  dice: ', scar_dices[-1])
        
        b  = 15
        bb = anno1.get_contour('lv_myo').envelope
        if hasattr(bb, 'exterior'):
            x, y = np.array(bb.exterior.xy)
            lx, ly, ux, uy = x.min()-b-10, y.min()-b, x.max()+b, y.max()+b
        fig, axes = plt.subplots(2,3, figsize=(22,15))
        for ax_ in axes: 
            for ax in ax_: 
                ax.imshow(img1, cmap='gray'); ax.axis('off')
                if hasattr(bb, 'exterior'):
                    ax.set_xlim([lx, ux]); ax.set_ylim([ly, uy])
        anno1.plot_contour_outlines(axes[0,0], 'lv_myo')
        anno1.plot_contour_face(    axes[1,0], 'saEnhancementReferenceMyoContour', c='r')
        anno1.plot_cont_comparison (axes[0,1], anno2, 'lv_myo')
        anno2.plot_contour_outlines(axes[0,2], 'lv_myo')
        anno2.plot_contour_face(    axes[1,2], 'saEnhancementReferenceMyoContour', c='r')
        anno1.plot_contour_outlines(axes[1,0], 'lv_myo')
        anno1.plot_contour_outlines(axes[1,0], 'scar_fwhm_excluded_area')
        anno1.plot_cont_comparison (axes[1,1], anno2, 'scar_fwhm_excluded_area')
        anno2.plot_contour_outlines(axes[1,2], 'lv_myo')
        anno2.plot_contour_outlines(axes[1,2], 'scar_fwhm_excluded_area')
        fig.tight_layout()
        #fig.savefig(os.path.join(bp, 'Outputs', case1.case_name+'_depth_'+str(d)+'.png'), dpi=300)
        plt.show()
    #break

In [ ]:
areas1 = [float("{:2.1f}".format(a)) for a in np.asarray(areas1)]
areas2 = [float("{:2.1f}".format(a)) for a in np.asarray(areas2)]
scar_dices_ = [scar_dices[i] for i in range(len(scar_dices)) if scar_dices[i]!=0]# and areas1[i]>1 and areas2[i]>1]
fractions   = [v for v in np.asarray(areas2)/np.asarray(areas1) if v!=np.inf and not np.isnan(v)]
print(len(scar_dices), len(scar_dices_))
print(len(fractions))
print(np.corrcoef(areas1, areas2))
print()

print(np.asarray(myo_dices).mean(), np.asarray(myo_dices).std())
print(np.asarray(scar_dices_).mean(), np.asarray(scar_dices_).std())
#print(np.asarray(areas1))
#print(np.asarray(areas2))

#print(areas1)
#print(areas2)
#print(np.asarray(areas2))
#print(fractions)

In [ ]:
import seaborn as sns
def scar_myo_relevance_plot(cases1, cases2):
    data = {'scar_dices':[], 'myo_dices':[], 'sf_diff':[]}
    for c1,c2 in zip(cases1, cases2):
        cat1, cat2 = c1.categories[0], c2.categories[0]
        for d in range(cat1.nr_slices):
            anno1 = cat1.get_anno(d,0)
            anno2 = cat2.get_anno(d,0)
            data['myo_dices'].append(dice(anno1.get_contour('lv_myo'), anno2.get_contour('lv_myo')))
            data['scar_dices'].append(dice(anno1.get_contour('scar_fwhm_excluded_area'), anno2.get_contour('scar_fwhm_excluded_area')))
            sf1 = anno1.get_contour('scar_fwhm_excluded_area').area / (anno1.get_contour('lv_myo').area+10**-9)
            sf2 = anno2.get_contour('scar_fwhm_excluded_area').area / (anno2.get_contour('lv_myo').area+10**-9)
            sf_diff = np.abs(sf1 - sf2)
            #sf_diff = 0.5 * (sf1 + sf2)
            data['sf_diff'].append(sf_diff)
    ax = sns.scatterplot(x='myo_dices', y='scar_dices', data=data, size='sf_diff')
    plt.show()
    
scar_myo_relevance_plot(cases1, cases2)

In [ ]:
def scar_fraction_plot(cases1, cases2):
    data = {'sf_diff':[], 'sf_avg':[], 'hue': []}
    for c1,c2 in zip(cases1, cases2):
        cat1, cat2 = c1.categories[0], c2.categories[0]
        base_pos, apex_pos = cat1.get_base_apex('lv_myo')
        for d in range(cat1.nr_slices):
            anno1 = cat1.get_anno(d,0)
            anno2 = cat2.get_anno(d,0)
            if not anno1.has_contour('lv_myo') or not anno2.has_contour('lv_myo'): continue
            if   d==base_pos: data['hue'].append(0)
            elif d==apex_pos: data['hue'].append(2)
            else            : data['hue'].append(1)
            sv1 = anno1.get_contour('scar_fwhm_excluded_area').area
            m1  = anno1.get_contour('lv_myo').area
            sv2 = anno2.get_contour('scar_fwhm_excluded_area').area
            m2  = anno2.get_contour('lv_myo').area
            sf1, sf2 = sv1/(m1+10**-9), sv2/(m2+10**-9)
            data['sf_diff'].append(sf1-sf2)
            data['sf_avg' ].append((sf1+sf2)/2.0)
    fig, ax = plt.subplots(1,1,figsize=(10,10))
    ax = sns.scatterplot(x='sf_avg', y='sf_diff', data=data, size='sf_avg', hue='hue')
    ax.set_title('Bland Altman SF')
    plt.show()
    
scar_fraction_plot(cases1, cases2)

In [ ]:
def scar_myo_relevance_plot(cases1, cases2):
    data = {'scar_dice':[], 'scar_vol_diff':[], 'hue': [], 'avg_vol': []}
    for c1,c2 in zip(cases1, cases2):
        cat1, cat2 = c1.categories[0], c2.categories[0]
        voxel_size = cat1.pixel_w*cat1.pixel_h*cat.spacing_between_slices
        base_pos, apex_pos = cat1.get_base_apex('scar_fwhm_excluded_area')
        for d in range(cat1.nr_slices):
            anno1 = cat1.get_anno(d,0)
            anno2 = cat2.get_anno(d,0)
            if not anno1.has_contour('scar_fwhm_excluded_area') or not anno2.has_contour('scar_fwhm_excluded_area'): continue
            if   d==base_pos: data['hue'].append(0)
            elif d==apex_pos: data['hue'].append(2)
            else            : data['hue'].append(1)
            scar1 = anno1.get_contour('scar_fwhm_excluded_area')
            scar2 = anno2.get_contour('scar_fwhm_excluded_area')
            data['scar_dice'    ].append(dice(scar1, scar2))
            data['scar_vol_diff'].append(voxel_size*(scar1.area-scar2.area)/1000.0)
            data['avg_vol'      ].append(voxel_size*(scar1.area+scar2.area)/1000.0/2)
    fig, ax = plt.subplots(1,1,figsize=(10,10))
    ax = sns.scatterplot(x='scar_vol_diff', y='scar_dice', data=data, size='avg_vol', hue='hue')
    ax.set_title('Bland Altman SF')
    plt.show()
    
scar_myo_relevance_plot(cases1, cases2)